In [3]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt

--2022-12-29 06:48:20--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110515 (108K) [text/plain]
Saving to: ‘dev_reviews.txt’

dev_reviews.txt     100%[===================>] 107.92K  --.-KB/s    in 0.005s  

2022-12-29 06:48:20 (22.0 MB/s) - ‘dev_reviews.txt’ saved [110515/110515]



## Приведем тестовые данные к нужному формату

In [1]:
import pandas as pd

In [2]:
dev_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id','text'])

In [3]:
dev_texts.head()

,text_id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [75]:
sentences_info = []
for row in dev_texts.itertuples():
    for sentence in sent_tokenize(row.text):
      s = []
      for token in word_tokenize(sentence):
        if token == "``" or token == "''":
          token = '"'
        s.append([row.text_id, token, row.text.index(token), row.text.index(token) + len(token), None])
      sentences_info.append(s)

In [7]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [9]:
def word2features(sentence, index):
    word = sentence[index][0]
    features = {'word.lower()': word.lower(), 'word.isupper()': word.isupper(), 'pos': str(morph.parse(word)[0].tag.POS), 'BOS': False}
    if index > 0:
      pass
    else:
        features['BOS'] = True
    if index < len(sentence) - 1:
      pass
    else:
        features['EOS'] = True
    return features

def sent2features(sentence):
    return [word2features(sentence, index) for index in range(len(sentence))]

def sent2labels(sentence):
    return [label for label, token in sentence]

In [41]:
import copy

In [42]:
dev_copy = copy.deepcopy(sentences_info)

In [46]:
# чистим данные от пока не нужных нам id текста, начала и конца
for i in range(len(dev_copy)):
  for k in range(len(dev_copy[i])):
    dev_copy[i][k].pop(0)
    dev_copy[i][k].pop(1)
    dev_copy[i][k].pop(1)
    dev_copy[i][k].pop(1)

In [10]:
dev = [sent2features(s) for s in dev_copy]

## Загрузим модель и сделаем предсказания

In [13]:
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 25.9 MB/s 


In [15]:
import sklearn_crfsuite

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pickle

In [18]:
crf = pickle.load(open('/content/drive/My Drive/crf.sav', 'rb'))

In [19]:
y_pred = crf.predict(dev)

In [268]:
results = []
for i in range(len(sentences_info)):
  for k in range(len(sentences_info[i])):
    results.append([sentences_info[i][k][0], y_pred[i][k], sentences_info[i][k][1], sentences_info[i][k][2], sentences_info[i][k][3], sentences_info[i][k][4]])

In [57]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt

--2022-12-29 07:38:20--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt [following]
--2022-12-29 07:38:20--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228391 (223K) [text/plain]
Saving to: ‘train_aspects.txt’

train_aspects.txt   100%[===================>] 223.04K  --.-KB/s    in 0.005s  

2022-12-29 07:38:20 (44.2 MB/s) - ‘train_aspects.txt’ save

## Добавляем информацию о тональности из train_aspects

In [58]:
train_aspects = pd.read_csv('train_aspects.txt', delimiter='\t', names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment'])

In [85]:
aspects_info = []
for row in train_aspects.itertuples():
    if len(row.mention.split()) == 1:
         aspects_info.append({'text_id': row.text_id, 'mention': row.mention, 'sentiment': row.sentiment})
    else:
        aspects_info.append({'text_id': row.text_id, 'mention': row.mention.split()[0], 'sentiment': row.sentiment})
        for i in range(1, len(row.mention.split())):
            aspects_info.append({'text_id': row.text_id, 'mention': row.mention.split()[i], 'sentiment': row.sentiment})

In [86]:
aspects_info[0]

{'text_id': 3976, 'mention': 'ресторане', 'sentiment': 'neutral'}

In [269]:
for result in results:
    for aspect in aspects_info:
        if aspect['text_id'] == result[0] and aspect['mention'] == result[2]:
            result[5] = aspect['sentiment']

## Конкатенируем n-граммы

In [308]:
results_labels_without_O = []
for i in range(len(results)):
  if results[i][1] != 'O':
    results_labels_without_O.append(results[i])

In [230]:
len(results_labels_without_O)

1486

In [271]:
results_labels_concatenated = []

In [272]:
results_labels_concatenated.append(results_labels_without_O[3])
results_labels_concatenated.append(results_labels_without_O[4])
results_labels_concatenated.append(results_labels_without_O[5])

In [273]:
results_labels_concatenated

[[13823, 'B-Service', 'проводила', 188, 197, 'neutral'],
 [13823, 'I-Service', 'к', 69, 70, 'neutral'],
 [13823, 'I-Service', 'столу', 200, 205, 'neutral']]

In [315]:
results_labels_concatenated = []
for i in range(len(results_labels_without_O)):
  try:
    if results_labels_without_O[i][1].startswith('B-') and results_labels_without_O[i + 1][1].startswith('B-'):
      results_labels_concatenated.append([results_labels_without_O[i][0], results_labels_without_O[i][1].split('-')[1], results_labels_without_O[i][2], results_labels_without_O[i][3], results_labels_without_O[i][4], results_labels_without_O[i][5]])
    else:
      n_gram = []
      n_gram.append(results_labels_without_O[i][2])
      k = i
      while results_labels_without_O[k + 1][1].startswith('I-'):
        n_gram.append(results_labels_without_O[k + 1][2])
        k += 1
      results_labels_concatenated.append([results_labels_without_O[i][0], results_labels_without_O[i][1].split('-')[1], ' '.join(n_gram), results_labels_without_O[i][3], results_labels_without_O[k + 1][4], results_labels_concatenated[i][5]])
  except(IndexError):
    pass

In [305]:
import csv

In [318]:
with open('aspects_сrf.tsv', 'w') as output:
  tsv_writer = csv.writer(output)
  for i in range(len(results_labels_concatenated)):
    row = '\t'.join(str(v) for v in results_labels_concatenated[i])
    tsv_writer.writerow([row])

## Оценка качества

In [323]:
gold_test_path = "dev_aspects.txt"
pred_test_path = "/content/aspects_сrf.txt"

In [320]:
from collections import defaultdict

In [321]:
gold_aspect_cats = {}
with open(gold_test_path) as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

In [324]:
full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open(pred_test_path) as fp:
    for line in fp:    
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i], 
                            doc_gold_aspect_cats["ends"][i], 
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i], 
                                    doc_gold_aspect_cats["ends"][i], 
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break

In [325]:
gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])

In [326]:
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7921225382932167
Full match recall: 0.6084033613445378
Partial match ratio in pred: 0.8676148796498906
Full category accuracy: 0.7724288840262582
Partial category accuracy: 0.8588621444201313

